In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#pd.set_option('max_columns',100)

import matplotlib as mpl
mpl.style.use('ggplot')
sns.set_style('white')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

import sys
import os
import tempfile
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import accuracy_score
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.metrics import (precision_score, recall_score,f1_score)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from bayes_opt import BayesianOptimization

In [3]:
dev_df = pd.read_csv('dev_df_revise.csv')
val_df = pd.read_csv('val_df_revise.csv')

In [4]:
# features = [
# 'hospstay_seq',
#  'icustay_rank',
#  'previous_mdrneg_lessthan90d',
#  'potassium',
#  'bicarbonate',
#  'creatinine',
#  'po2',
#  'eosinophils',
#  'monocytes_abs',
#  'lymphocytes',
#  'ptt',
#  'hematocrit',
#  'mch',
#  'mchc',
#  'platelet',
#  'rbc',
#  'rdw',
#  'history_dialysis',
#  'charlson_comorbidity_index',
#  'ant_duration',
#  'ant_1stday',
#  'immu_1stday',
#  'ent_nut',

#   'CEFTRIAXONE_anttest_beforeicu90d',
#   'TRIMETHOPRIM.SULFA_anttest_beforeicu90d',
#   'CEFEPIME_anttest_beforeicu90d'
#   'CEFTAZIDIME_anttest_beforeicu90d',
#   'CIPROFLOXACIN_anttest_beforeicu90d',
#   'PIPERACILLIN.TAZO_anttest_beforeicu90d',
#   'MEROPENEM_anttest_beforeicu90d',
#   'GENTAMICIN_anttest_beforeicu90d',
#   'TOBRAMYCIN_anttest_beforeicu90d'
# ]
# labels = ['CEFEPIME', 'CEFTAZIDIME', 'CIPROFLOXACIN','PIPERACILLIN.TAZO', 'MEROPENEM','GENTAMICIN', 'TOBRAMYCIN']

In [5]:
label_features_dict = {
'stay_chart_included_mdr':['dementia',
 'platelet',
 'phosphate',
 'heart_rate',
 'weight',
 'creatinine',
 'basophils_abs',
 'lymphocytes_abs',
 'icustay_rank',
 'temperature',
 'sodium',
 'history_difficulty_swallowing',
 'los_hosp_beforeicu',
 'hospstay_seq',
 'neutrophils_abs',
 'wbc',
 'mg',
 'NLR',
 'admission_age',
 'history_assistive_devices_Wheelchair',
 'mchc',
 'history_past_COPD',
 'bun',
 'rdw',
 'previous_mdr_90d'
]
}

In [6]:
#gbdt
def gbdt_cv(n_estimators, min_samples_split, max_features, max_depth):
    res = cross_val_score( 
        GradientBoostingClassifier(n_estimators=int(n_estimators),
                                                        min_samples_split=int(min_samples_split),
                                                        max_features=min(max_features, 0.999), # float
                                                        max_depth=int(max_depth),
                                                        random_state=2023
        ),
        x_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return res

gbdt_op = BayesianOptimization(
        gbdt_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)},
    random_state=2023
)

#random forest
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    res = cross_val_score( 
        RandomForestClassifier(n_estimators=int(n_estimators),
                            min_samples_split=int(min_samples_split),
                            max_features=min(max_features, 0.999), # float
                            max_depth=int(max_depth),
                            random_state=2023
        ),
        x_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return res

rf_op = BayesianOptimization(
        rf_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (3, 15)},
    random_state = 2023
)

#xgboost
def xgb_cv(n_estimators, min_child_weight,  subsample, colsample_bytree, max_depth):
    res = cross_val_score(
        XGBClassifier(n_estimators=int(n_estimators),
                            min_child_weight=int(min_child_weight),
                            #gama = min(gama,0.999),
                            subsample = min(subsample,0.999),
                            colsample_bytree = min(colsample_bytree,0.999),
                            max_depth=int(max_depth),
                            learning_rate = 0.01,
                            #n_jobs = -1,
                            random_state=2023
        ),
        x_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return res

xgb_op = BayesianOptimization(
        xgb_cv,
        {'n_estimators': (10, 250),
        'min_child_weight': (2, 25),
        #'gama': (0.1, 0.999),
        'subsample': (0.1, 0.999),
        'colsample_bytree': (0.1, 0.999),
        'max_depth': (3, 15)},
    random_state = 2023
)

In [7]:
def optimize_rf(n_iter):
    total_param_dict = {}
    models_dict = {}
    
    def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
        return cross_val_score( 
                RandomForestClassifier(n_estimators=int(n_estimators),
                                    min_samples_split=int(min_samples_split),
                                    max_features=min(max_features, 0.999), # float
                                    max_depth=int(max_depth),
                                    random_state=2023
                ),
                x_train, y_train, scoring='roc_auc', cv=5
            ).mean()

    for i in label_features_dict:
        label = i
        features = label_features_dict[i]
        x_train = dev_df[features]
        y_train = dev_df[label]
        x_test = val_df[features]
        y_test = val_df[label]        
        
        rf_op = BayesianOptimization(
            rf_cv,
            {'n_estimators': (10, 300),
            'min_samples_split': (2, 25),
            'max_features': (0.1, 0.999),
            'max_depth': (3, 20)},
        random_state = 2023)

        print('---------------------------特征为：%s-------------------------------'%label)
        print('---------------------------模型为：Random Forest-------------------------------')
        rf_op.maximize(n_iter=n_iter)
        print(rf_op.max)
        models_dict[label] = rf_op.max
        print('----------------------------------------------------------------------------------------------------')
        
    return models_dict
    
def optimize_xgb(n_iter):
    total_param_dict = {}
    models_dict = {}
    
    def xgb_cv(n_estimators, min_child_weight,  subsample, colsample_bytree, max_depth):
        res = cross_val_score(
            XGBClassifier(n_estimators=int(n_estimators),
                                min_child_weight=int(min_child_weight),
                                #gama = min(gama,0.999),
                                subsample = min(subsample,0.999),
                                colsample_bytree = min(colsample_bytree,0.999),
                                max_depth=int(max_depth),
                                learning_rate = 0.01,
                                #n_jobs = -1,
                                random_state=2023
            ),
            x_train, y_train, scoring='roc_auc', cv=5
        ).mean()
        return res

    for i in label_features_dict:
        label = i
        features = label_features_dict[i]
        x_train = dev_df[features]
        y_train = dev_df[label]
        x_test = val_df[features]
        y_test = val_df[label]        
        
        xgb_op = BayesianOptimization(
                xgb_cv,
                {'n_estimators': (10, 250),
                'min_child_weight': (2, 25),
                #'gama': (0.1, 0.999),
                'subsample': (0.1, 0.999),
                'colsample_bytree': (0.1, 0.999),
                'max_depth': (3, 15)},
            random_state = 2023
        )

        print('---------------------------特征为：%s-------------------------------'%label)
        print('---------------------------模型为：XGBOOST-------------------------------')
        xgb_op.maximize(n_iter=n_iter)
        print(xgb_op.max)
        models_dict[label] = xgb_op.max
        print('----------------------------------------------------------------------------------------------------')
        
    return models_dict

In [8]:
models_dict_rf = optimize_rf(45)

---------------------------特征为：stay_chart_included_mdr-------------------------------
---------------------------模型为：Random Forest-------------------------------
|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.6939    | 8.474     | 0.9005    | 15.53     | 46.71     |
| 2         | 0.6943    | 5.403     | 0.5206    | 2.508     | 220.9     |
| 3         | 0.6982    | 11.91     | 0.5899    | 12.5      | 155.4     |
| 4         | 0.7009    | 9.706     | 0.2359    | 10.3      | 57.0      |
| 5         | 0.6945    | 8.745     | 0.2621    | 10.99     | 20.34     |
| 6         | 0.7034    | 8.893     | 0.2191    | 9.434     | 299.6     |
| 7         | 0.6983    | 9.99      | 0.3619    | 10.89     | 56.68     |
| 8         | 0.7005    | 9.308     | 0.5131    | 10.17     | 299.8     |
| 9         | 0.7003    | 8.767     | 0.4431    | 9.378     | 298.5     |
| 10        | 0.7047    

In [9]:
models_dict_xgb = optimize_xgb(45)

---------------------------特征为：stay_chart_included_mdr-------------------------------
---------------------------模型为：XGBOOST-------------------------------
|   iter    |  target   | colsam... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------
| 1         | 0.6986    | 0.3895    | 13.69     | 15.53     | 40.38     | 0.2271    |
| 2         | 0.6961    | 0.5206    | 3.265     | 18.73     | 135.9     | 0.5899    |
| 3         | 0.7058    | 0.5103    | 9.017     | 11.07     | 46.28     | 0.4244    |
| 4         | 0.7009    | 0.2457    | 7.056     | 6.147     | 103.8     | 0.132     |
| 5         | 0.706     | 0.6078    | 5.442     | 9.374     | 100.4     | 0.2655    |
| 6         | 0.7035    | 0.864     | 11.17     | 3.412     | 196.4     | 0.5708    |
| 7         | 0.7036    | 0.3614    | 4.916     | 15.0      | 116.6     | 0.7846    |
| 8         | 0.705     | 0.8176    | 11.7      | 23.26     | 194.5   

In [10]:
models_dict_rf_mod = models_dict_rf.copy()
models_dict_xgb_mod =models_dict_xgb.copy()

In [11]:
for i in models_dict_rf_mod:
    models_dict_rf_mod[i]['params']['max_depth'] = int(models_dict_rf_mod[i]['params']['max_depth'])
    models_dict_rf_mod[i]['params']['max_features'] = round(models_dict_rf_mod[i]['params']['max_features'],3)
    models_dict_rf_mod[i]['params']['min_samples_split'] = int(models_dict_rf_mod[i]['params']['min_samples_split'])
    models_dict_rf_mod[i]['params']['n_estimators'] = int(models_dict_rf_mod[i]['params']['n_estimators'])

In [12]:
for i in models_dict_xgb_mod:
    models_dict_xgb_mod[i]['params']['max_depth'] = int(models_dict_xgb_mod[i]['params']['max_depth'])
    models_dict_xgb_mod[i]['params']['colsample_bytree'] = round(models_dict_xgb_mod[i]['params']['colsample_bytree'],3)
    models_dict_xgb_mod[i]['params']['subsample'] = round(models_dict_xgb_mod[i]['params']['subsample'],3)
    models_dict_xgb_mod[i]['params']['min_child_weight'] = int(models_dict_xgb_mod[i]['params']['min_child_weight'])
    models_dict_xgb_mod[i]['params']['n_estimators'] = int(models_dict_xgb_mod[i]['params']['n_estimators'])

In [13]:
np.save('rf_params_revise.npy',models_dict_rf_mod)
np.save('xgb_params_revise.npy',models_dict_xgb_mod)

In [14]:
#models_dict_rf_mod = np.load('rf_params.npy',allow_pickle=True).item()

In [15]:
# total_param_dict = {}
# models_dict = {}
# n_iter = 195
# for i in label_features_dict:
#     label = i
#     features = label_features_dict[i]
#     x_train = dev_df[features]
#     y_train = dev_df[label]
#     x_test = val_df[features]
#     y_test = val_df[label]
    
# #     gbdt_op = BayesianOptimization(
# #         gbdt_cv,
# #         {'n_estimators': (10, 250),
# #         'min_samples_split': (2, 25),
# #         'max_features': (0.1, 0.999),
# #         'max_depth': (5, 15)},
# #     random_state=2023)

#     rf_op = BayesianOptimization(
#         rf_cv,
#         {'n_estimators': (10, 250),
#         'min_samples_split': (2, 25),
#         'max_features': (0.1, 0.999),
#         'max_depth': (3, 15)},
#     random_state = 2023)
    
#     xgb_op = BayesianOptimization(
#         xgb_cv,
#         {'n_estimators': (10, 250),
#         'min_child_weight': (2, 25),
#         #'gama': (0.1, 0.999),
#         'subsample': (0.1, 0.999),
#         'colsample_bytree': (0.1, 0.999),
#         'max_depth': (3, 15)},
#     random_state = 2023)
    
#     print('---------------------------特征为：%s-------------------------------'%label)
# #     print('---------------------------模型为：GBDT-------------------------------')
# #     gbdt_op.maximize(n_iter=n_iter)
# #     print(gbdt_op.max)
# #     models_dict['gbdt'] = gbdt_op.max
#     print('---------------------------模型为：Random Forest-------------------------------')
#     rf_op.maximize(n_iter=n_iter)
#     print(rf_op.max)
#     models_dict['rf'] = rf_op.max
#     print('---------------------------模型为：XGBOOST-------------------------------')
#     xgb_op.maximize(n_iter=n_iter)
#     print(xgb_op.max)
#     models_dict['xgboost'] = xgb_op.max
#     total_param_dict[label] = models_dict
#     print('----------------------------------------------------------------------------------------------------')